In [1]:
!pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/a4/e1/ce276f553811bd6c684cfe5f637a33ae6444750746f974a8f73d5dc92004/chromadb-0.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/e2/03/f3c8ba0a6b6e30d7d18c40faab90807c9bb5e9a1e3b2fe2008af624a9c97/build-1.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/d2/32/a91850c7aa8a34f61838913155103808fe90da6f1ea4302731b59e9ba6f2/chroma_hnswlib-0.7.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/e6/33/de41e554e5a187d583906e10d53bfae5fd6c07e98cbf4fe5262bd37e739a/fastapi-0.111.0-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/b2/f9/e6f30ba6094733e4f9794fd098ca0543a19b07ac1fa

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.2 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.3 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


In [2]:
!pip install faiss-cpu

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip install pandas numpy torch transformers chromadb

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb
from chromadb.config import Settings

def main():
    print("Loading model and tokenizer...")
    # Load the model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
    model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

    # Function to get embeddings in batches
    def get_embeddings(texts, batch_size=32):
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
            with torch.no_grad():
                outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            embeddings.append(batch_embeddings)
            print(f"Processed batch {i//batch_size + 1}/{(len(texts) + batch_size - 1)//batch_size}")
        return np.vstack(embeddings)

    print("Loading dataset...")

    # Upload dataset to Colab
    from google.colab import files
    uploaded = files.upload()

    # Load dataset from CSV
    df = pd.read_csv(next(iter(uploaded)))

    # Check if 'Context' and 'Response' columns exist
    if 'Context' not in df.columns or 'Response' not in df.columns:
        raise ValueError("CSV file must contain 'Context' and 'Response' columns.")

    # Extract 'context' and 'response' columns
    contexts = df['Context'].tolist()
    responses = df['Response'].tolist()

    print("Getting embeddings for the contexts...")
    # Get embeddings for the contexts
    context_embeddings = get_embeddings(contexts, batch_size=32)

    print(f"Context embeddings shape: {context_embeddings.shape}")

    # Initialize ChromaDB client with SQLite persistence
    settings = Settings(persist_directory="./chromadb_store")
    client = chromadb.Client(settings=settings)

    try:
        print("Creating collection 'context_response_store'...")
        collection = client.create_collection("context_response_store")
        print("Collection 'context_response_store' created.")
    except ValueError:
        print("Collection 'context_response_store' already exists, fetching the collection...")
        collection = client.get_collection("context_response_store")
        print("Fetched existing collection 'context_response_store'.")

    print("Saving contexts, responses, and embeddings to ChromaDB...")
    # Prepare data to add to ChromaDB
    ids = [str(i) for i in range(len(contexts))]
    embeddings = [embedding.tolist() for embedding in context_embeddings]

    # Add data to ChromaDB collection
    collection.add(
        ids=ids,
        embeddings=embeddings,
        metadatas=[{"context": context, "response": response} for context, response in zip(contexts, responses)]
    )

    print("Vector store created successfully.")

    # Verify persistence by listing all collections
    print("Listing all collections...")
    collections = client.list_collections()
    print("Available collections:", collections)

if __name__ == "__main__":
    main()



Loading model and tokenizer...


ValueError: Unrecognized model identifier in sentence-transformers/all-MiniLM-L6-v2. Should contains one of 'bert', 'openai-gpt', 'gpt2', 'transfo-xl', 'xlnet', 'xlm', 'roberta', 'ctrl'

In [5]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import chromadb
from chromadb.config import Settings

# Load the model and tokenizer
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Initialize ChromaDB client with SQLite persistence
settings = Settings(persist_directory="./chromadb_store")
client = chromadb.Client(settings=settings)

# Ensure the collection exists
try:
    print("Fetching collection 'context_response_store'...")
    collection = client.get_collection("context_response_store")
    print("Collection 'context_response_store' retrieved successfully.")
except ValueError:
    raise ValueError("Collection 'context_response_store' does not exist. Make sure to run create_vector_store.py first.")

# Function to get embeddings for a list of texts
def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

# Function to retrieve top k responses based on similar context
def retrieve_responses(query, collection, k=3):
    query_embedding = get_embeddings([query])[0]
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=k
    )
    print("Query results:", results)  # Debugging line
    similar_responses = [
        {"Context": metadata['context'], "Response": metadata['response']}
        for metadata_list in results["metadatas"] for metadata in metadata_list
    ]
    return similar_responses

Loading model and tokenizer...


ValueError: Unrecognized model identifier in sentence-transformers/all-MiniLM-L6-v2. Should contains one of 'bert', 'openai-gpt', 'gpt2', 'transfo-xl', 'xlnet', 'xlm', 'roberta', 'ctrl'

In [6]:
class PromptTemplate:
    def __init__(self, similarity_search):
        self.similarity_search = similarity_search

    def one_shot(self, query):
        examples = self.similarity_search(query, num_examples=1)
        return self._format_prompt(query, examples)

    def two_shot(self, query):
        examples = self.similarity_search(query, num_examples=2)
        return self._format_prompt(query, examples)

    def few_shot(self, query):
        examples = self.similarity_search(query, num_examples=3)
        return self._format_prompt(query, examples)

    def _format_prompt(self, query, examples):
        formatted_examples = "\n".join(
            [f"Example {i+1}:\nContext: {ex['Context']}\nResponse: {ex['Response']}\n" for i, ex in enumerate(examples)]
        )
        return f"Query: {query}\n\n{formatted_examples}"


In [7]:
# Create an instance of the PromptTemplate class with the retrieve_responses function
prompt_template = PromptTemplate(lambda query, num_examples: retrieve_responses(query, collection, k=num_examples))

# Test the one_shot, two_shot, and few_shot methods
query = "I am feeling lonely"
one_shot_result = prompt_template.one_shot(query)
two_shot_result = prompt_template.two_shot(query)
few_shot_result = prompt_template.few_shot(query)

print("\nOne-shot result:\n", one_shot_result)
print("\nTwo-shot result:\n", two_shot_result)
print("\nFew-shot result:\n", few_shot_result)

NameError: name 'retrieve_responses' is not defined